In [6]:
# import libraries
import numpy as np
import torch
import torch.nn.functional as F
from tslearn.preprocessing import TimeSeriesScalerMinMax
from tqdm import tqdm

class LSTM(torch.nn.Module):
    def __init__(self, output_size, n_feat, hidden_dim, n_layers):
        super(LSTM, self).__init__()
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        self.lstm = torch.nn.LSTM(n_feat, hidden_dim, n_layers)
        self.fc = torch.nn.Linear(hidden_dim, output_size)
        
    def forward(self, x, hidden, batch_size):
        lstm_out, hidden = self.lstm(x, hidden)
        out = self.fc(lstm_out)
        return out, hidden
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device))
        return hidden
        

In [7]:
# Load Data 
cluster_range = 12
seq_len = 12
lstm_mse = []
epochs = 100
data_path = '/Users/muneeza/Documents/GitHub/DATA_SMest/'

# Set description variables
sub_feat_names = ['AREA', 'PRECIP' , 'ET', 'SW_END', 'PERC', 'GW_RCHG', 'DA_RCHG', 'REVAP', 'SA_IRR', 'DA_IRR', 'SA_ST', 'DA_ST',
                 'WYLD', 'DAILYCN', 'TMP_AV', 'SOL_TMP', 'SOLAR']

train_id_en = 27*12
test_id_en = 34*12 

for cluster in range(cluster_range):
    # Read data for the cluster -  (timeseries, hrus , features)
    data = np.load(data_path+'sub_hru_sub_data_clstr_'+str(cluster)+'.npy') # (tstep , hrus, features)

    # Select id for target variable 
    target_id = sub_feat_names.index('SW_END')


    # Test Train Split 
    X_train = data[ 0:train_id_en, : ,  :]
    X_test = data[ train_id_en: test_id_en , : ,  :]

    # Normalize data
    X_train = TimeSeriesScalerMinMax(value_range=(-1,1)).fit_transform( np.transpose(X_train,(1,0,2)) )
    X_test = TimeSeriesScalerMinMax(value_range=(-1,1)).fit_transform( np.transpose(X_test,(1,0,2)) )
    X_train = np.transpose(X_train, (1,0,2) ) # reshape back to original 
    X_test = np.transpose(X_test, (1,0,2) ) # reshape back to original 


    # Convert to supervised learneing example, target is t+1

    n_feat = len(sub_feat_names)
    n_hrus = X_train.shape[1]
    test_len = X_test.shape[0]

    Y_train = np.zeros((train_id_en - seq_len , n_hrus, seq_len))
    Y_test = np.zeros((test_len - seq_len , n_hrus, seq_len))

    for i in range(1 , train_id_en - seq_len):
        Y_train[i, : , :] = X_train[i:i+seq_len  , : , target_id ].T

    for i in range(1 , test_len - seq_len):    
        Y_test[i, : , :] = X_test[i:i+seq_len , : , target_id].T
        

    X_train = X_train[0:train_id_en-seq_len , : , :  ]
    X_test = X_test[0:test_len-seq_len , : , : ]

    # Set up model for training 

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = LSTM(output_size=12 , n_feat= n_feat, hidden_dim=32, n_layers=1)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    model.train()
    hist = []
    train_dataset = torch.tensor(X_train).double()
    model = model.double()
    batch_size = X_train.shape[1]
    for ep in tqdm(range(epochs)):
        cost = torch.tensor([0.0],requires_grad=True)
        hidden = model.init_hidden(batch_size = batch_size)
        y_hat = model(train_dataset, hidden, batch_size=batch_size)
        cost = cost + torch.mean((y_hat[0].data-Y_train)**2)
        cost = cost / (ep+1)
        hist.append(cost.item())
        cost.backward()
        optimizer.step()
        optimizer.zero_grad()

    # Performance on test data  
    model.eval()
    test_dataset = torch.tensor(X_test).double()
    cost = 0
    batch_size = test_dataset.shape[1]
    hidden = model.init_hidden(batch_size = batch_size)
    y_hat = model(test_dataset, hidden, batch_size=batch_size)
    cost = cost + torch.mean((y_hat[0].data-Y_test)**2)
    lstm_mse.append(cost.item())
    print("MSE: {:.4f}".format(lstm_mse[cluster]))

100%|██████████| 100/100 [01:17<00:00,  1.28it/s]


MSE: 0.3433


100%|██████████| 100/100 [00:34<00:00,  2.93it/s]


MSE: 0.3815


100%|██████████| 100/100 [00:34<00:00,  2.86it/s]


...

100%|██████████| 100/100 [00:50<00:00,  1.99it/s]


MSE: 0.3057


100%|██████████| 100/100 [00:39<00:00,  2.51it/s]


MSE: 0.3677


  4%|▍         | 4/100 [00:03<01:31,  1.04it/s]